In [23]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import  accuracy_score
from sklearn.metrics import  classification_report
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer


In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Create an instance of TfidfVectorizer
vectorizer = TfidfVectorizer()

# Example text documents
documents = [
    "This is the first document.",
    "This document is the second document.",
    "And this is the third one.",
    "Is this the first document?"
]

# Fit and transform the documents
tfidf_matrix = vectorizer.fit_transform(documents)

# Get the feature names (words)
feature_names = vectorizer.get_feature_names()

# Print the feature names and TF-IDF matrix
print("Feature Names:", feature_names)
print("TF-IDF Matrix:")
print(tfidf_matrix.toarray())


Feature Names: ['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third', 'this']
TF-IDF Matrix:
[[0.         0.46979139 0.58028582 0.38408524 0.         0.
  0.38408524 0.         0.38408524]
 [0.         0.6876236  0.         0.28108867 0.         0.53864762
  0.28108867 0.         0.28108867]
 [0.51184851 0.         0.         0.26710379 0.51184851 0.
  0.26710379 0.51184851 0.26710379]
 [0.         0.46979139 0.58028582 0.38408524 0.         0.
  0.38408524 0.         0.38408524]]


c:\Users\Lakshya\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [28]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Lakshya\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [29]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

<h4>Data Pre processing :</h4>

In [30]:
df_fake = pd.read_csv("G:\Fake news analysis\Dataset\Fake.csv")
df_true = pd.read_csv("G:\Fake news analysis\Dataset\True.csv")


In [42]:
df_fake.head()

,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


In [32]:
df_true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [33]:
df_fake.shape, df_true.shape

((23481, 4), (21417, 4))

<h4>Inserting column 'label' as target feature</h4>

In [34]:
df_fake['label'] = 0
df_true['label'] = 1

<u><b>conclusion : </b></U>new column is created named 'label'.

In [35]:
df_fake.shape, df_true.shape

((23481, 5), (21417, 5))

In [38]:
# counting the number for missing values in the fake news dataset 
df_fake.isnull().sum()

title      0
text       0
subject    0
date       0
label      0
dtype: int64

In [43]:
# counting the number for missing values in the true news dataset 
df_fake.isnull().sum()

title      0
text       0
subject    0
date       0
label      0
dtype: int64

<u><b>conclusion: </b></U>there is no missing values are present. 

Merging Two Dataframes:

In [46]:
df_merge = pd.concat([df_fake,df_true], axis=0)
df_merge.tail()

,title,text,subject,date,label
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017",1
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017",1
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017",1
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017",1
21416,Indonesia to buy $1.14 billion worth of Russia...,JAKARTA (Reuters) - Indonesia will buy 11 Sukh...,worldnews,"August 22, 2017",1


In [47]:
df_merge.columns

Index(['title', 'text', 'subject', 'date', 'label'], dtype='object')

it is that the title can provide some initial insights, but it may be designed to be attention-grabbing or provocative without reflecting the accuracy of the article's content.
so more reliable is to focus on analysing the content of news rather than title.

In [49]:
# dropping un-necessary column
# axis=0 would mean dropping rows.
# axis=1 means dropping columns.

df = df_merge.drop(['title','subject','date'],axis=1)

In [52]:
df.head()

,text,label
0,Donald Trump just couldn t wish all Americans ...,0
1,House Intelligence Committee Chairman Devin Nu...,0
2,"On Friday, it was revealed that former Milwauk...",0
3,"On Christmas day, Donald Trump announced that ...",0
4,Pope Francis used his annual Christmas Day mes...,0


In [53]:
# shuffell the data frame

In [55]:
df = df.sample(frac = 1)
df

,text,label
12571,The woman s name is Denise Scott. You can dona...,0
16753,"We can t say for sure, but we re guessing the ...",0
20154,LUANDA (Reuters) - Angola s Constitutional Cou...,1
2254,A White House press release which congratulate...,0
16118,Susan Rice just announced she won t testify be...,0
...,...,...
149,The Attorney General of the United States lied...,0
5746,"DOVER AIR FORCE BASE, Del. (Reuters) - The U....",1
18884,OSLO (Reuters) - Myanmar s leader Aung San Suu...,1
2718,WASHINGTON (Reuters) - A U.S. federal appeals ...,1


In [56]:
df.reset_index(inplace = True)


In [64]:
df.columns



Index(['text', 'label'], dtype='object')

defining dependent and independent variable

In [65]:
X = df['text']
Y = df['label']

porter stemming : reduces words to their root word or base word.
example- word = easily, root word = easy

In [71]:
port_stem = PorterStemmer()

In [72]:
# defining a function
def stemming (content):
    # [^a-zA-Z] removes all part like no. or code or any numerical data, punctuatuion marks and all except the alphabets
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)  
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    # removing all stopwords if present
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content
    

In [75]:
df['text'] = df['text'].apply(stemming)